In [1]:
import torch 
from torch import nn

#from models import VisualEncoder
from train import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
"""self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )"""

'self.encoder = VisualEncoder()\n        self.encoder.load_state_dict(\n            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device(\'cpu\'))\n        )'

In [3]:
env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
env.update_taskset(TaskSet(preset=['C17']))
env = PovOnlyWrapper(env)
env = IgluActionWrapper(env)

In [4]:
obs = env.reset()

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/root/miniconda/envs/py37/lib/python3.7/runpy.py:125: RuntimeWarning: 'minerl_patched.utils.process_watcher' found in sys.modules after import of package 'minerl_patched.utils', but prior to execution of 'minerl_patched.utils.process_watcher'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))


In [5]:
action = env.action_space.sample()
obs, reward, done, info = env.step(action)

In [11]:
info['target_grid'].shape

(9, 11, 11)

In [7]:
from torch.nn.functional import one_hot

target = info['target_grid']
target = one_hot(torch.tensor(target, dtype=torch.long), num_classes=7).permute(3, 0, 1, 2).float()

In [8]:
target.shape

torch.Size([7, 9, 11, 11])

In [9]:
img = torch.tensor(obs.copy(), dtype=torch.float).permute(2, 0, 1) / 255.0

In [10]:
img.shape

torch.Size([3, 64, 64])

In [12]:
visual_encoder = VisualEncoder()
img_features = visual_encoder(img[None,:])

In [13]:
img_features.shape

torch.Size([1, 512])

In [14]:
linear_1 = nn.Linear(512, 7)

In [15]:
Q = linear_1(img_features)

In [16]:
conv_1 = nn.Conv3d(7, 7, 1)

In [17]:
V = conv_1(target[None,:])
V.shape

torch.Size([1, 7, 9, 11, 11])

In [18]:
Q[0].shape, V[0].shape

(torch.Size([7]), torch.Size([7, 9, 11, 11]))

In [20]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import numpy as np
from typing import Optional, Tuple

class ScaledDotProductAttention(nn.Module):
    """
    Scaled Dot-Product Attention proposed in "Attention Is All You Need"
    Compute the dot products of the query with all keys, divide each by sqrt(dim),
    and apply a softmax function to obtain the weights on the values
    Args: dim, mask
        dim (int): dimention of attention
        mask (torch.Tensor): tensor containing indices to be masked
    Inputs: query, key, value, mask
        - **query** (batch, q_len, d_model): tensor containing projection vector for decoder.
        - **key** (batch, k_len, d_model): tensor containing projection vector for encoder.
        - **value** (batch, v_len, d_model): tensor containing features of the encoded input sequence.
        - **mask** (-): tensor containing indices to be masked
    Returns: context, attn
        - **context**: tensor containing the context vector from attention mechanism.
        - **attn**: tensor containing the attention (alignment) from the encoder outputs.
    """
    def __init__(self, dim: int):
        super(ScaledDotProductAttention, self).__init__()
        self.sqrt_dim = np.sqrt(dim)

    def forward(self, query: Tensor, key: Tensor, value: Tensor, mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor]:
        score = torch.bmm(query, key.transpose(1, 2)) / self.sqrt_dim

        if mask is not None:
            score.masked_fill_(mask.view(score.size()), -float('Inf'))

        attn = F.softmax(score, -1)
        context = torch.bmm(attn, value)
        return context, attn

In [27]:
key = value = V.reshape(-1, 7, 9*11*11).permute(0, 2, 1)
value.shape

torch.Size([1, 1089, 7])

In [28]:
query = Q.reshape(-1, 1, 7)
query.shape

torch.Size([1, 1, 7])

In [29]:
attention = ScaledDotProductAttention(7)

In [34]:
attention(query, key, value)[1].shape

torch.Size([1, 1, 1089])

In [64]:
"""
    Q: (bs, q_length=ch)
    K, V: (bs, ch, heigth, width, depth)
"""

Q[0] @ V.permute(0, 2, 3, 4, 1)[0]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (693x11 and 7x1)

In [63]:
V.permute(0, 2, 3, 4, 1).shape

torch.Size([1, 9, 11, 11, 7])

In [58]:
7 * 9 * 11

693

In [54]:
cross_attention(Q, V, V)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (693x11 and 7x1)

In [49]:
Q.shape

torch.Size([1, 7])

In [52]:
def cross_attention(Q, K, V):
    """
    Q: (bs, q_length=ch)
    K, V: (bs, ch, heigth, width, depth)
    """
    scores = Q @ K # (bs, 1, height, width, depth)
    scores = nn.functional.softmax(scores, dim=1)
    result = torch.matmul(scores.view(0, 2, 3, 4, 1), V.view(0, 2, 3, 4, 1)).view(0, 4, 1, 2, 3)
    return result

In [ ]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        
        self.visual_encoder = VisualEncoder()
        self.visual_dim = 512
        
        self.conv_1 = nn.Conv3d(7, 7, 1)
        self.linear_1 = nn.Linear(self.visual_dim, 7)
        
        
        

In [ ]:
def scaled_dot_product_attention(self, Q, K, V):
    batch_size = Q.size(0)
    k_length = K.size(-2)

    # Scaling by d_k so that the soft(arg)max doesnt saturate
    Q = Q / np.sqrt(self.d_k)  # (bs, n_heads, q_length, dim_per_head)
    scores = torch.matmul(Q, K.transpose(2,3))  # (bs, n_heads, q_length, k_length)

    A = nn_Softargmax(dim=-1)(scores)  # (bs, n_heads, q_length, k_length)

    # Get the weighted average of the values
    H = torch.matmul(A, V)  # (bs, n_heads, q_length, dim_per_head)

    return H, A